In [6]:
import pandas as pd

import sqlite3
import csv

In [7]:
cbb = pd.read_csv('CSV_Files/cbb.csv')
cbb21 = pd.read_csv('CSV_Files/cbb21.csv')

In [8]:
cbb

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450,Michigan St.,B10,35,26,111.4,87.8,0.9392,50.6,44.5,20.8,...,32.4,50.4,44.3,34.1,30.1,64.4,6.7,S16,3.0,2013
2451,Arizona,P12,35,27,114.4,92.2,0.9229,52.5,46.6,19.5,...,32.9,50.6,43.4,37.1,35.8,66.8,4.6,S16,6.0,2013
2452,Oregon,P12,37,28,104.8,88.6,0.8728,49.3,46.4,21.4,...,33.3,49.1,44.9,33.3,33.4,69.2,2.9,S16,12.0,2013
2453,La Salle,A10,34,24,112.0,96.2,0.8516,51.9,49.3,17.1,...,28.5,49.3,50.6,37.7,30.2,66.0,0.3,S16,13.0,2013


In [15]:
cbb21

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,SEED
0,Michigan,B10,24,20,118.1,91.1,0.9521,54.9,44.9,16.3,...,24.8,28.9,24.5,53.3,42.3,38.7,33.5,66.9,7.2,1.0
1,Baylor,B12,24,22,123.2,94.5,0.9548,57.5,49.1,17.6,...,30.9,27.0,31.7,54.1,48.1,41.8,34.0,68.8,6.6,1.0
2,Illinois,B10,29,23,117.7,90.4,0.9539,55.6,46.6,18.2,...,22.2,39.2,30.5,55.3,45.4,37.6,32.7,70.7,8.9,1.0
3,Gonzaga,WCC,26,26,125.4,89.8,0.9791,61.0,47.5,16.1,...,23.4,36.7,25.9,64.0,46.8,36.5,32.5,74.6,8.5,1.0
4,Iowa,B10,29,21,123.5,95.7,0.9491,54.6,48.3,13.3,...,28.6,32.0,22.6,52.4,45.8,38.6,34.8,70.0,5.6,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,Louisiana Tech,CUSA,27,21,102.7,93.4,0.7479,50.5,45.6,18.4,...,23.4,35.3,26.4,49.7,46.4,34.6,29.6,69.6,-1.7,NaN
343,Toledo,MAC,29,21,113.3,101.8,0.7743,54.3,48.3,15.5,...,30.7,28.5,23.4,52.5,51.0,37.7,29.5,69.1,-2.1,NaN
344,UAB,CUSA,27,22,102.5,94.6,0.7153,48.6,47.0,15.6,...,25.4,29.9,27.1,49.4,46.1,31.1,32.4,67.5,-2.7,NaN
345,Eastern Kentucky,OVC,27,22,101.5,102.3,0.4749,51.2,51.0,16.6,...,30.2,25.6,32.2,49.6,51.8,35.8,33.0,75.1,-3.8,NaN


In [9]:
# replacing the postseason with an easy to read solution
# the number refers to what round each team reached

final_round = {
    'R64' : 1,
    'R32' : 2,
    'S16' : 3,
    'E8'  : 4,
    'F4'  : 5,
    '2ND' : 6,
    'Champions' : 7,
    None : 0,
    'R68' : 0
}

cbb['POSTSEASON'] = cbb['POSTSEASON'].replace(final_round)

In [16]:
# replacing the postseason with an easy to read solution
# the number refers to what round each team reached

Seeding = {
    None : 0
}

cbb21['SEED'] = cbb21['SEED'].replace(Seeding)

In [10]:
# renaming columns for sql purposes

cbb.columns = ['TEAM', 'CONF', 'G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', 'Two_P_O', 'Two_P_D', 'Three_P_O',
       'Three_P_D', 'ADJ_T', 'WAB', 'POSTSEASON', 'SEED', 'YEAR']

### Using SQLite for the CBB dataset with all the teams from 2013-2019

In [27]:
con = sqlite3.connect('SQL_dbs/cbb.db')

cur = con.cursor()

In [28]:
cbb.to_sql('cbb', con, if_exists = 'replace', index = False)

In [29]:
cbb_teams = pd.read_sql("""SELECT * 
                         FROM cbb 
                         WHERE postseason >= 1""", con)

In [30]:
Avg_by_round = pd.read_sql("""SELECT AVG(adjoe), AVG(adjde), AVG(barthag), AVG(efg_o), AVG(efg_d), AVG(tor), 
                      AVG(tord), AVG(orb), AVG(drb), AVG(ftr), AVG(ftrd), AVG(two_p_o), 
                      AVG(two_p_d), AVG(three_p_o), AVG(three_p_d), AVG(adj_t), AVG(wab), postseason
               FROM cbb
               WHERE postseason >= 1
               GROUP BY postseason""", con)

### Using SQLite for the CBB21 dataset with all the teams from the 2021 season

In [18]:
con2 = sqlite3.connect('SQL_dbs/cbb21.db')
cur2 = con2.cursor()

cbb21.to_sql('cbb21', con2, if_exists = 'replace', index = False)

In [24]:
cbb21_top_68 = pd.read_sql("""SELECT * 
                              FROM cbb21 
                              WHERE seed >= 1""", con2)

In [25]:
cbb21_top_68

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,SEED
0,Michigan,B10,24,20,118.1,91.1,0.9521,54.9,44.9,16.3,...,24.8,28.9,24.5,53.3,42.3,38.7,33.5,66.9,7.2,1.0
1,Baylor,B12,24,22,123.2,94.5,0.9548,57.5,49.1,17.6,...,30.9,27.0,31.7,54.1,48.1,41.8,34.0,68.8,6.6,1.0
2,Illinois,B10,29,23,117.7,90.4,0.9539,55.6,46.6,18.2,...,22.2,39.2,30.5,55.3,45.4,37.6,32.7,70.7,8.9,1.0
3,Gonzaga,WCC,26,26,125.4,89.8,0.9791,61.0,47.5,16.1,...,23.4,36.7,25.9,64.0,46.8,36.5,32.5,74.6,8.5,1.0
4,Iowa,B10,29,21,123.5,95.7,0.9491,54.6,48.3,13.3,...,28.6,32.0,22.6,52.4,45.8,38.6,34.8,70.0,5.6,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Mount St. Mary's,NEC,22,12,95.7,100.6,0.3593,48.7,45.5,20.1,...,24.6,29.7,24.6,47.7,45.6,33.7,30.1,62.2,-7.2,16.0
64,Hartford,AE,23,15,97.3,99.6,0.4345,50.4,47.6,19.0,...,28.6,28.6,25.0,51.3,50.7,32.7,28.7,66.9,-4.9,16.0
65,Norfolk St.,MEAC,21,16,99.4,104.1,0.3698,49.7,47.4,17.8,...,28.7,38.6,42.0,46.1,47.2,37.4,31.7,67.9,-5.0,16.0
66,Texas Southern,SWAC,23,16,97.3,103.5,0.3307,48.3,46.5,21.4,...,25.7,38.9,33.8,51.0,45.2,27.7,32.4,71.8,-4.5,16.0


In [32]:
cbb_teams.to_csv('CSV_Files/cbb_teams.csv')
Avg_by_round.to_csv('CSV_Files/avg_by_round.csv')
cbb21_top_68.to_csv('CSV_Files/cbb21_top_68.csv')